Import necessary files

In [ ]:
!pip install neuron
from neuron import h
import matplotlib.pyplot as plt
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 13.0 MB/s eta 0:00:00


Functions that create a symmetric branched dendrite with constant length following Rall's Law. Number of offshoots per branch is determined by branch_num, and depth of branch iteration is branch_iter

In [ ]:
def gen_dend(dends, string, parent = None, branch_num = None):
  dend = h.Section(name= string + '_' + str(len(dends)))
  dend.nseg=11
  #need to change dend.L too to fit Rall's Law
  dend.diam = 1
  dend.Ra = 100
  dend.cm = 1
  dend.insert('pas')
  dend.g_pas = 0.001
  dend.e_pas = -65
  if parent is not None:
    if string == 'basal' and parent.name() == 'soma_0':
      dend.connect(parent(0))
    else:
      dend.connect(parent(1))
    if branch_num is not None:
      #Diameter adjustment makes passive branches follow Rall's Law.
      dend.diam = parent.diam / (branch_num ** (2 / 3))
  #Length algorithm makes all sections have an electrotonic length of 1
  dend.L = (((dend.diam * 2500) / (dend.g_pas * dend.Ra)) ** 0.5) * 1
  dends.append(dend)

def gen_tree(dends, string, parent, branch_num, branch_iter):
  if branch_iter > 0:
    for i in range (branch_num):
      gen_dend(dends, string, parent, branch_num)
      gen_tree(dends, string, dends[-1], branch_num, branch_iter - 1)

In [ ]:
# h.load_file('stdrun.hoc')

# dends = []
# #Initialize a section serving as the trunk or soma of this tree
# #merge 1J graphs
# #Also apical and basal
# #input files (conductances and synapses)
# gen_dend(dends)

# #Create a tree off of the previously initialized trunk. CHANGE THE FOLLOWING VARS TO ALTER TREE MORPHOLOGY
# branch_num = 2
# branch_iter = 3
# gen_tree(dends, dends[0], branch_num, branch_iter)
# gen_tree(dends, dends[0], branch_num, branch_iter, 'basal')

# h.topology()

Place synapses randomly across sections of choice

In [ ]:
def place_syns(dends, synapse_list):
  syn_list = []
  for i in synapse_list:
    #seg_placement = random.random()  # x is a float in [0.0, 1.0)
    #syn = h.Exp2Syn(dends[i](seg_placement))
    syn = h.Exp2Syn(dends[i.branch_id](i.segment))
    syn.tau1 = 0.2  # rise
    syn.tau2 = 1.5  # decay
    syn.e = 0
    syn_list.append(syn)

  # Create a presynaptic spike generator
  stim = h.NetStim()
  stim.number = 1       # number of spikes
  stim.start = 5        # time (ms) of the spike
  stim.interval = 10    # irrelevant for 1 spike
  stim.noise = 0

  # Connect stim to synapse
  nc_list = []
  for i in syn_list:
    nc = h.NetCon(stim, i)
    nc.delay = 0          # optional delay (ms)
    nc.weight[0] = 0.001  # synaptic conductance in µS. CHANGE THIS TO SHOW SUBLINEARITY
    nc_list.append(nc)
  return syn_list, stim, nc_list

# syn_indeces = [4, 7]   #CHANGE THIS TO CONTROL WHICH SECTIONS HAVE SYNAPSES (4 -> dend4)
# syn_list, stim, nc_list = place_syns(dends, syn_indeces)

In [ ]:
# t_vec = h.Vector().record(h._ref_t)          # time vector
# v_vec = h.Vector().record(dends[0](0.5)._ref_v)  # voltage at trunk/soma center

# h.finitialize(-65)  # starting voltage in mV
# h.continuerun(40)   # ms

def autoscale(data, max = None):
    if max is None:
      max = np.max(data)
    if max == np.min(data):
        return np.zeros_like(data)
    return (data - np.min(data)) / (max - np.min(data))

def EPSP_vecs_sublinear(dends, synapse_list):
  syn_list, stim, nc_list = place_syns(dends, synapse_list)
  weight_unit = nc_list[0].weight[0]
  rev = -65
  maxes = []
  for i in range (5):
    v = h.Vector().record(dends[0](0.5)._ref_v)  # voltage at trunk/soma center
    h.finitialize(rev)
    while h.t < 100:
      h.fadvance()
    maxes.append(max(v) - rev)
    for nc in nc_list:
      nc.weight[0] += weight_unit
  return maxes

def EPSP_vecs_linear(dends, synapse_list):
  rev = -65
  maxes = []
  for i in range (1, len(synapse_list) + 1):
    syn_list, stim, nc_list = place_syns(dends, synapse_list[0:i])
    v = h.Vector().record(dends[0](0.5)._ref_v)  # voltage at trunk/soma center
    h.finitialize(rev)
    while h.t < 100:
      h.fadvance()
    maxes.append(max(v) - rev)
  return maxes

def arithmetic(maxes):
  arith = []
  for i in range(1, len(maxes) + 1):
    arith.append(maxes[0] * i)
  return arith

# syn_num = 5 # CHANGE THIS TO ADJUST TOTAL OF SYNAPSES ADDED
# maxes_0 = EPSP_vecs_sublinear(dends, [0], syn_num)
# arith_0 = arithmetic(maxes_0)
# maxes_0 = autoscale(maxes_0, arith_0[-1])
# arith_0 = autoscale(arith_0)
# maxes_1 = EPSP_vecs_sublinear(dends, [1], syn_num)
# arith_1 = arithmetic(maxes_1)
# maxes_1 = autoscale(maxes_1, arith_1[-1])
# arith_1 = autoscale(arith_1)
# maxes_2 = EPSP_vecs_sublinear(dends, [2], syn_num)
# arith_2 = arithmetic(maxes_2)
# maxes_2 = autoscale(maxes_2, arith_2[-1])
# arith_2 = autoscale(arith_2)
# maxes_3 = EPSP_vecs_sublinear(dends, [3], syn_num)
# arith_3 = arithmetic(maxes_3)
# maxes_3 = autoscale(maxes_3, arith_3[-1])
# arith_3 = autoscale(arith_3)

# lin_maxes = EPSP_vecs_linear(dends, [3, 6, 10, 13, 4, 7, 11, 14])
# lin_arith = arithmetic(lin_maxes)

# maxes_basal = EPSP_vecs_sublinear(dends, [15], syn_num)
# arith_basal = arithmetic(maxes_basal)

# def plot_volts(maxes, arith, string):

# plt.figure(figsize=(6, 4))
# plt.plot(arith_0, maxes_0, '-o', label='Section 0')
# plt.plot(arith_1, maxes_1, '-o', label='Section 1')
# plt.plot(arith_2, maxes_2, '-o', label='Section 2')
# plt.plot(arith_3, maxes_3, '-o', label='Section 3')
# x = np.linspace(0, 1, 100)
# plt.plot(x, x, '--', color='grey', label='y = x')
# plt.xlabel('Predicted EPSP (Normalized)')
# plt.ylabel('Actual EPSP (Normalized)')
# plt.title(f'Soma Membrane Potential')
# plt.xlim(0, 1)
# plt.ylim(0, 1)
# plt.legend(loc='upper left')
# plt.grid(True)
# plt.show()

# plot_volts(maxes_0, arith_0, '(Section 0)')
# plot_volts(maxes_1, arith_1, '(Section 1)')
# plot_volts(maxes_2, arith_2, '(Section 2)')
# plot_volts(maxes_3, arith_3, '(Section 3)')
# plot_volts(lin_maxes, lin_arith, '(Sections 3, 6, 10, 13 Added Respectively)')
# plot_volts(maxes_basal, arith_basal, 'Basal')